In [1]:
# !pip install scikit-learn kfp numpy
import pandas as pd 
import pickle
import tensorflow as tf
import numpy as np 
import time
import os

from kfp import components

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

2024-07-09 19:15:08.431360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 19:15:08.576590: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-07-09 19:15:08.576738: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-07-09 19:15:08.652795: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.

KeyboardInterrupt



In [3]:
def read_file() -> None:
    import os
    import pandas as pd
    import numpy as np
    from minio import Minio
    from scipy.special import boxcox
    from sklearn.model_selection import train_test_split
    import boto3
    import json
    
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine
    
    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    def zscore_normalization(df, name):
        mean = df[name].mean()
        sd = df[name].std()
        df[name] = (df[name] - mean) / sd
    def preprocess(df):
        #df = df.drop(columns=['Name', 'md5'])
        for i in df.columns:
            if i != 'outcome':
                #convert data to fit normal distribution
                df[i] = boxcox(df[i], 0.5)
                #normalize all numerical columns
                zscore_normalization(df, i)
        correlation_matrix = df.corr()
        cols_to_drop = []
        for i in df.columns:
            for j in df.columns:
                #drop columns with low correlation to target variable
                if i != j and i != 'outcome' and j != 'outcome' and abs(correlation_matrix[i][j]) > 0.6 and i not in cols_to_drop and j not in cols_to_drop:
                    cols_to_drop.append(i)
        cols_to_drop = set(cols_to_drop)
        df.drop(columns=cols_to_drop, inplace=True)
        return df

    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }

    
    # Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()

    # Query to fetch data from the table
    try:
        fetch_query = "SELECT * FROM outcomes join malware_data on malware_data.uid::text=outcomes.uid::text"
        df = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    cols = [col for col in df.columns if col != 'uid']
    
    #print(cols)
    df = df[cols]
    df = df.drop(columns=['datatype'])
    df = preprocess(df)
    
    
    X = df.drop(columns=['outcome'])
    y = df['outcome']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    
    bucket_name="mlpipelineews"
    role_arn = 'arn:aws:iam::533267059960:role/aws-s3-access'
    session_name = 'kubeflow-pipeline-session'
    sts_client = boto3.client('sts')
    response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName=session_name)
    credentials = response['Credentials']
    # Configure AWS SDK with temporary credentials
    s3_client = boto3.client('s3',
                      aws_access_key_id=credentials['AccessKeyId'],
                      aws_secret_access_key=credentials['SecretAccessKey'],
                      aws_session_token=credentials['SessionToken'])
    
    
    
    folder_path = './tmp/malware'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")
        

    df.to_csv("./tmp/malware/malware_data.csv")
    s3_client.upload_file("./tmp/malware/malware_data.csv", bucket_name, "malware_dataset.csv")
    np.save("./tmp/malware/X_train.npy",X_train)
    s3_client.upload_file("./tmp/malware/X_train.npy", bucket_name, "X_train.npy")
    np.save("./tmp/malware/y_train.npy",y_train)
    s3_client.upload_file("./tmp/malware/y_train.npy", bucket_name, "y_train.npy")
    np.save("./tmp/malware/X_test.npy",X_test)
    s3_client.upload_file("./tmp/malware/X_test.npy", bucket_name, "X_test.npy")
    np.save("./tmp/malware/y_test.npy",y_test)
    s3_client.upload_file("./tmp/malware/y_test.npy", bucket_name, "y_test.npy")


In [4]:
read_file()

Connected to PostgreSQL successfully.


/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Folder './tmp/malware' already exists.


In [6]:
def train_op() -> None:
    import pickle
    import pandas as pd
    import numpy as np
    import json
    import os
    import time
    import tensorflow as tf
    import boto3
    from minio import Minio
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sqlalchemy import create_engine
    from sqlalchemy import create_engine, Table, Column, Float, Integer, String, MetaData, ARRAY
    from sqlalchemy import select, desc, insert, text
    from io import BytesIO
    
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine
    
    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    bucket_name="mlpipelineews"
    role_arn = 'arn:aws:iam::533267059960:role/aws-s3-access'
    session_name = 'kubeflow-pipeline-session'
    sts_client = boto3.client('sts')
    response = sts_client.assume_role(RoleArn=role_arn, RoleSessionName=session_name)
    credentials = response['Credentials']
    
    
    # Configure AWS SDK with temporary credentials
    s3_client = boto3.client('s3',
                      aws_access_key_id=credentials['AccessKeyId'],
                      aws_secret_access_key=credentials['SecretAccessKey'],
                      aws_session_token=credentials['SessionToken'])
    
    
    response = s3_client.get_object(Bucket=bucket_name, Key="X_train.npy")
    data = response['Body'].read()
    X_train = np.load(BytesIO(data))

    
    response = s3_client.get_object(Bucket=bucket_name, Key="y_train.npy")
    data = response['Body'].read()
    y_train = np.load(BytesIO(data))

    
    response = s3_client.get_object(Bucket=bucket_name, Key="X_test.npy")
    data = response['Body'].read()
    X_test = np.load(BytesIO(data))

    
    response = s3_client.get_object(Bucket=bucket_name, Key="y_test.npy")
    data = response['Body'].read()
    y_test = np.load(BytesIO(data))
    
    
    # Define dataframe to store model metrics
    metrics = pd.DataFrame(columns=["Version", "Model", "Accuracy", "F1", "Precision", "Recall", "Train_Time", "Test_Time"])
    models_path = './tmp/malware/models'
    
    if not os.path.exists(models_path):
        os.makedirs(models_path)
        print(f"Folder '{models_path}' created successfully.")
    else:
        print(f"Folder '{models_path}' already exists.")
        
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
        
        

    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
    
    # Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()
        
    # Query to fetch data from the table
    try:
        fetch_query = "SELECT * FROM model_metrics ORDER BY created_at DESC LIMIT 1;"
        df = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    if(not df.empty):
        version = df['version'] + 1
    else:
        version = 1
        
    folder_path = f"version{version}"
    
    cursor.close()
    conn.close()

    
    
    #Logistic Regression
    start_train = time.time()
    lrc = LogisticRegression(random_state=0, max_iter=1000)
    lrc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    ypredlr = lrc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test, ypredlr)
    f1 = f1_score(y_test, ypredlr)
    precision = precision_score(y_test, ypredlr)
    recall = recall_score(y_test, ypredlr)
    metrics.loc[len(metrics.index)] = [version,'lrc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    with open('./tmp/malware/models/lrc.pkl', 'wb') as f:
        pickle.dump(lrc, f)
    s3_client.upload_file("tmp/malware/models/lrc.pkl", bucket_name, f"{folder_path}/lrc/model.pkl")
    
    
    #Random Forest Classifier
    start_train = time.time()
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred2=rfc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test, y_pred2)
    f1 = f1_score(y_test, y_pred2)
    precision = precision_score(y_test, y_pred2)
    recall = recall_score(y_test, y_pred2)
    metrics.loc[len(metrics.index)] = [version, 'rfc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    with open('./tmp/malware/models/rfc.pkl', 'wb') as f:
        pickle.dump(rfc, f)
    s3_client.upload_file("tmp/malware/models/rfc.pkl", bucket_name, f"{folder_path}/rfc/model.pkl")
    
    
    #Decision Tree
    start_train = time.time()
    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred3=dtc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred3)
    f1 = f1_score(y_test, y_pred3)
    precision = precision_score(y_test, y_pred3)
    recall = recall_score(y_test, y_pred3)
    metrics.loc[len(metrics.index)] = [version, 'dtc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    with open('./tmp/malware/models/dtc.pkl', 'wb') as f:
        pickle.dump(dtc, f)
    s3_client.upload_file("tmp/malware/models/dtc.pkl", bucket_name, f"{folder_path}/dtc/model.pkl")
    
    
    
    #Support Vector Machine
    start_train = time.time()
    svc = SVC()
    svc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred4=svc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred4)
    f1 = f1_score(y_test,y_pred4)
    precision = precision_score(y_test, y_pred4)
    recall = recall_score(y_test, y_pred4)
    metrics.loc[len(metrics.index)] = [version, 'svc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    with open('./tmp/malware/models/svc.pkl', 'wb') as f:
        pickle.dump(svc, f)
    s3_client.upload_file("tmp/malware/models/svc.pkl", bucket_name, f"{folder_path}/svc/model.pkl")
    
    
    
    #Gradient Boost
    start_train = time.time()
    gbc = GradientBoostingClassifier()
    gbc.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred5=gbc.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred5)
    f1 = f1_score(y_test, y_pred5)
    precision = precision_score(y_test, y_pred5)
    recall = (recall_score(y_test, y_pred5))
    metrics.loc[len(metrics.index)] = [version, 'gbc', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]
    with open('./tmp/malware/models/gbc.pkl', 'wb') as f:
        pickle.dump(gbc, f)
    s3_client.upload_file("tmp/malware/models/gbc.pkl", bucket_name, f"{folder_path}/gbc/model.pkl")
    
    
    #Gaussian Naive Bayes
    start_train = time.time()
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    end_train = time.time()
    start_test = time.time()
    y_pred6=gnb.predict(X_test)
    end_test = time.time()
    accuracy = accuracy_score(y_test,y_pred6)
    f1 = f1_score(y_test, y_pred6)
    precision = precision_score(y_test,y_pred6)
    recall = recall_score(y_test, y_pred6)
    metrics.loc[len(metrics.index)] = [version, 'gnb', accuracy, f1, precision, recall, end_train-start_train, end_test-start_test]  
    with open('./tmp/malware/models/gnb.pkl', 'wb') as f:
        pickle.dump(gnb, f)      
    s3_client.upload_file("tmp/malware/models/gnb.pkl", bucket_name, f"{folder_path}/gnb/model.pkl")
    
    
    
    #Artificial Neural Network
    input_shape = [X_train.shape[1]]
    start_train = time.time()
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    model.build()
    model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=256, epochs=25)
    end_train=time.time()
    start_test = time.time()
    y_pred7 = model.predict(X_test)
    y_pred7 = (y_pred7 > 0.5).astype(np.int32)
    end_test = time.time()
    print(y_pred7)
    accuracy = accuracy_score(y_test,y_pred7)
    f1 = f1_score(y_test, y_pred7)
    precision = precision_score(y_test,y_pred7)
    recall = recall_score(y_test, y_pred7)
    # accuracy = history.history['accuracy'][11]
    metrics.loc[len(metrics.index)] = [version, 'ann', accuracy, f1, precision, recall, end_test-start_test, 0]
    with open('./tmp/malware/models/ann.pkl', 'wb') as f:
        pickle.dump(model, f)
    s3_client.upload_file("tmp/malware/models/ann.pkl", bucket_name, f"{folder_path}/ann/model.pkl")

    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
        
    insert_query = """
        INSERT INTO model_metrics (name, version, URI, in_use, accuracy, f1, precision, recall, train_time, test_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (name, version) DO NOTHING;
    """
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")

        # Iterate through DataFrame rows and insert into the table
        for index, row in metrics.iterrows():
            cursor.execute(insert_query, (
                row['Model'], 
                row['Version'], 
                f"s3://mlpipelineews/version{version}/{row['Model']}/model.pkl", 
                False, 
                row['Accuracy'], 
                row['F1'], 
                row['Precision'], 
                row['Recall'], 
                row['Train_Time'], 
                row['Test_Time']
            ))
    
        conn.commit()
        print("Data inserted successfully.")

        cursor.close()
        conn.close()
        print("PostgreSQL connection closed.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL or insert data: {e}")
    

In [7]:
train_op()

2024-07-10 20:04:21.342946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 20:04:21.562764: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-07-10 20:04:21.562919: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-07-10 20:04:21.589583: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.


Folder './tmp/malware/models' already exists.
Connected to PostgreSQL successfully.


/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2024-07-10 20:08:19.960961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 20:08:19.961194: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/25
432/432 [==============================] - 4s 7ms/step - loss: 0.0801 - accuracy: 0.9762 - val_loss: 0.0434 - val_accuracy: 0.9855
Epoch 2/25
432/432 [==============================] - 3s 6ms/step - loss: 0.0439 - accuracy: 0.9862 - val_loss: 0.0397 - val_accuracy: 0.9871
Epoch 3/25
432/432 [==============================] - 3s 6ms/step - loss: 0.0383 - accuracy: 0.9876 - val_loss: 0.0400 - val_accuracy: 0.9871
Epoch 4/25
432/432 [==============================] - 3s 6ms/step - loss: 0.0353 - accuracy: 0.9883 - val_loss: 0.0346 - val_accuracy: 0.9891
Epoch 5/25
432/432 [==============================] - 3s 6ms/step - loss: 0.0332 - accuracy: 0.9888 - val_loss: 0.0339 - val_accuracy: 0.9895
Epoch 6/25
432/432 [==============================] - 3s 6ms/step - loss: 0.0319 - accuracy: 0.9893 - val_loss: 0.0332 - val_accuracy: 0.9898
Epoch 7/25
432/432 [==============================] - 3s 6ms/step - loss: 0.0304 - accuracy: 0.9898 - val_loss: 0.0323 - val_accuracy: 0.9894
Epoch 

[I 240710 20:09:29 builder_impl:779] Assets written to: ram://8b27c8cf-3e49-45f7-8120-0e5b5cceae7b/assets


Connected to PostgreSQL successfully.
Data inserted successfully.
PostgreSQL connection closed.


In [30]:
def model_eval_deploy() -> None:
    import pickle
    import pandas as pd
    import numpy as np 
    import json
    import os 
    import time
    import tensorflow as tf
    
    import boto3
    
    import psycopg2
    from psycopg2 import sql
    from sqlalchemy import create_engine
    
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1SKLearnSpec
    
    def get_secret():

        secret_name = "DBCreds"
        region_name = "us-east-1"

        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            raise e

        secret = get_secret_value_response['SecretString']
    
        # Parse the secret string to get the credentials
        secret_dict = json.loads(secret)
        username = secret_dict['username']
        password = secret_dict['password']
        host = secret_dict['host']
        port = secret_dict['port']
        dbname = secret_dict['dbname']

        return username, password, host, port, dbname


    (user,pswd,host,port,db) = get_secret()
    
    
    db_details = {
        'dbname': db,
        'user': user,
        'password': pswd,
        'host': host,
        'port': port
    }
    
    # Connect to PostgreSQL
    try:
        conn = psycopg2.connect(**db_details)
        cursor = conn.cursor()
        print("Connected to PostgreSQL successfully.")
    except Exception as e:
        print(f"Failed to connect to PostgreSQL: {e}")
        exit()
        
    # Query to fetch data from the table
    try:
        fetch_query = "SELECT * FROM model_metrics ORDER BY created_at DESC LIMIT 1;"
        df = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    #print(f"df = {df['version'][0]}")
    
    if(not df.empty):
        version = df['version'][0]
    else:
        version = 1
        
    #print(f"version = {version}")
    
    try:
        fetch_query = f"SELECT * FROM model_metrics where version={version} order by accuracy desc limit 1;"
        model = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    
    name = f"{model['name'][0]}-version{version}"
    print(name)
    namespace = utils.get_default_target_namespace()
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version
    
    isvc2 = V1beta1InferenceService(
        api_version=api_version,
        kind=constants.KSERVE_KIND,
        metadata=client.V1ObjectMeta(
            name=name,
            namespace=namespace,
            annotations={'sidecar.istio.io/inject': 'false'}
        ),
        spec=V1beta1InferenceServiceSpec(
            predictor=V1beta1PredictorSpec(
                service_account_name="s3-service-account",
                sklearn=V1beta1SKLearnSpec(
                    storage_uri=model['uri'][0]
                )
            )
        )
    )

    
    KServe = KServeClient()
    KServe.create(isvc2)
    
    try:
        fetch_query = "SELECT * FROM model_metrics where in_use is true LIMIT 1;"
        old_model = pd.read_sql(fetch_query, conn)
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    update_query_new = """
        UPDATE model_metrics
        SET in_use = true
        WHERE name = %s and version = %s;
    """
    
    update_query_old = """
        UPDATE model_metrics
        SET version = false
        WHERE name = %s and version = %s;
    """
    try:
        cursor.execute(update_query_new, (model['name'][0], int(model['version'][0])))
        if(not old_model.empty):
            cursor.execute(update_query_old, (old_model['name'][0], int(old_model['version'][0])))
        conn.commit()
    except Exception as e:
        print(f"Failed to fetch data: {e}")
    
    if(not old_model.empty):
        del_name = f"{old_model['name']}-version{old_model['version']}"
        namespace = utils.get_default_target_namespace()

        # Initialize the KServe client
        KServe = KServeClient()

        # Delete the inference service
        KServe.delete(del_name, namespace)
    
    
    cursor.close()
    conn.close()

In [31]:
model_eval_deploy()

Connected to PostgreSQL successfully.
rfc-version1


/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
import kfp
from kfp import dsl
from kfp import components

read_csv_op = components.func_to_container_op(func=read_file, output_component_file='preprocess.yaml', base_image='python:3.7', packages_to_install=['pandas','scikit-learn==1.0.1', 'kfp', 'numpy', 'minio'])

train_op = components.func_to_container_op(func=train_op, output_component_file='train.yaml', base_image='python:3.7', packages_to_install=['pandas', 'scikit-learn==1.0.1','numpy','minio', 'tensorflow'])

read_data_op = kfp.components.load_component_from_file('preprocess.yaml')
train_op = kfp.components.load_component_from_file('train.yaml')

# @dsl.pipeline(
#     name='Machine Learning Pipeline',
#     description='A pipeline to preprocess, train, and predict using sklearn and tensorflow'
# )

def ml_pipeline():
    preprocess = read_csv_op()
    train = train_op().after(preprocess)

# Compile the pipeline
kfp.compiler.Compiler().compile(ml_pipeline, 'malware_pipeline.yaml')

In [ ]:
update_query_new = """
    UPDATE model_metrics
    SET in_use = true
    WHERE name = '%s' and version = %s;
"""